In [1]:
# coding:utf-8
import numpy as np
import random
import os
random.seed(0)
np.random.seed(0)#seed应该在main里尽早设置，以防万一
os.environ['PYTHONHASHSEED'] =str(0)#消除hash算法的随机性
import transformers as _
from transformers1 import Trainer, TrainingArguments,BertTokenizer
from NLP_Utils import MLM_Data,train_data,blockShuffleDataLoader

from NEZHA.configuration_nezha import NeZhaConfig
from NEZHA.modeling_nezha import NeZhaForMaskedLM

2022-10-01 00:47:52.843454: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-10-01 00:47:52.843493: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


训练集样本数量： 10000


In [2]:
maxlen=510
batch_size=16
file_dir = '/root/autodl-tmp/CCF-小样本/Nezha_pytorch/nezha_model/'
tokenizer = BertTokenizer.from_pretrained(file_dir)
config = NeZhaConfig.from_pretrained(file_dir)
# config = NeZhaConfig(
#     vocab_size=len(tokenizer),
#     hidden_size=768,
#     num_hidden_layers=12,
#     num_attention_heads=12,
#     max_position_embeddings=512,
# )

# nezha-cn-base初始权重需要从Github下载https://github.com/lonePatient/NeZha_Chinese_PyTorch
model = NeZhaForMaskedLM.from_pretrained(file_dir)

# model.resize_token_embeddings(len(tokenizer))
# # 只训练word_embedding。能缩短两倍的训练时间
# for name, p in model.named_parameters():
#     if name != 'bert.embeddings.word_embeddings.weight':
#         p.requires_grad = False
# print(model)
# print('train_data:', train_data)

train_MLM_data=MLM_Data(train_data, maxlen, tokenizer)

#自己定义dataloader，不要用huggingface的
dl=blockShuffleDataLoader(train_MLM_data,None,key=lambda x:len(x[0])+1,shuffle=False
                          ,batch_size=batch_size,collate_fn=train_MLM_data.collate)

You are using a model of type nezha to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at /root/autodl-tmp/CCF-小样本/Nezha_pytorch/nezha_model/ were not used when initializing NeZhaForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing NeZhaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NeZhaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
len(dl), len(train_MLM_data)

(625, 10000)

In [7]:
for i in dl:
    print(i)
    break

{'input_ids': tensor([[ 101, 3315, 4509,  ...,    0,    0,    0],
        [ 101, 3315, 1355,  ...,    0,    0,    0],
        [ 101,  103, 1355,  ...,    0,    0,    0],
        ...,
        [ 101, 3315, 2916,  ...,    0,    0,    0],
        [ 101,  103, 2141,  ...,    0,    0,    0],
        [ 101, 3315,  103,  ...,    0,    0,    0]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[ True,  True,  True,  ..., False, False, False],
        [ True,  True,  True,  ..., False, False, False],
        [ True,  True,  True,  ..., False, False, False],
        ...,
        [ True,  True,  True,  ..., False, False, False],
        [ True,  True,  True,  ..., False, False, False],
        [ True,  True,  True,  ..., False, False, False]], device='